# Multilingual Semantics Probe

## Step 1: Corpus Generation

In [131]:
from __future__ import annotations

import itertools
import json
from dataclasses import dataclass
from typing import Dict, List

import pandas as pd
import os

In [132]:
STIMULI_DIR = "./stimuli"

if not os.path.exists(STIMULI_DIR):
    os.mkdir(STIMULI_DIR)

In [133]:
# --- English lexicon ---
EN_SUBJECTS = [
    "shark",
    "robot",
    "chef",
    "dog",
]

EN_OBJECTS = [
    "pirate",
    "student",
    "doctor",
    "tourist",
]

# Use correct simple past forms
EN_VERBS_PAST = [
    "ate",
    "helped",
    "pushed",
    "chased",
]

# --- Mandarin lexicon ---
# Bare nouns only (no quantifiers inside)
ZH_SUBJECTS = [
    "鲨鱼",
    "机器人",
    "厨师",
    "狗",
]

ZH_OBJECTS = [
    "海盗",
    "学生",
    "医生",
    "游客",
]

# Verb stems compatible with 了
ZH_VERBS = [
    "吃",
    "帮助",
    "推",
    "追",
]

# Optional classifier map (defaults to 个)
ZH_CLASSIFIER: Dict[str, str] = {
    "鲨鱼": "只",
    "狗": "只",
    "机器人": "个",
    "厨师": "个",
    "海盗": "个",
    "学生": "个",
    "医生": "个",
    "游客": "个",
}

In [134]:
EN_TEMPLATES = [
    # Classic ambiguous English form
    "A {subj} {verb_past} every {obj}.",
]

ZH_TEMPLATES = [
    # Canonical Mandarin surface-scope reading
    "有一{cl}{subj}{verb}了每个{obj}。",
]

In [135]:
@dataclass(frozen=True)
class Stimulus:
    language: str
    template_id: str
    subj: str
    obj: str
    verb: str
    sentence: str


def get_classifier(noun: str, cl_map: Dict[str, str]) -> str:
    return cl_map.get(noun, "个")


def generate_english(
    subjects: List[str],
    objects: List[str],
    verbs_past: List[str],
) -> List[Stimulus]:
    out: List[Stimulus] = []
    for tid, tmpl in enumerate(EN_TEMPLATES):
        for subj, obj, verb in itertools.product(subjects, objects, verbs_past):
            out.append(
                Stimulus(
                    language="en",
                    template_id=f"en_{tid}",
                    subj=subj,
                    obj=obj,
                    verb=verb,
                    sentence=tmpl.format(
                        subj=subj,
                        obj=obj,
                        verb_past=verb,
                    ),
                )
            )
    return out


def generate_mandarin(
    subjects: List[str],
    objects: List[str],
    verbs: List[str],
    cl_map: Dict[str, str],
) -> List[Stimulus]:
    out: List[Stimulus] = []
    for tid, tmpl in enumerate(ZH_TEMPLATES):
        for subj, obj, verb in itertools.product(subjects, objects, verbs):
            cl = get_classifier(subj, cl_map)
            out.append(
                Stimulus(
                    language="zh",
                    template_id=f"zh_{tid}",
                    subj=subj,
                    obj=obj,
                    verb=verb,
                    sentence=tmpl.format(
                        cl=cl,
                        subj=subj,
                        obj=obj,
                        verb=verb,
                    ),
                )
            )
    return out

In [ ]:
stimuli = []
stimuli += generate_english(EN_SUBJECTS, EN_OBJECTS, EN_VERBS_PAST)
stimuli += generate_mandarin(ZH_SUBJECTS, ZH_OBJECTS, ZH_VERBS, ZH_CLASSIFIER)

continuation_df = pd.DataFrame([s.__dict__ for s in stimuli])

# Stable IDs for downstream scoring
continuation_df.insert(
    0,
    "stimulus_id",
    [
        f"{row.language}-{row.template_id}-{row.Index:06d}"
        for row in continuation_df.itertuples()
    ],
)

In [ ]:
print("Total stimuli:", len(continuation_df))
print(continuation_df["language"].value_counts())

display(
    continuation_df[continuation_df["language"] == "en"][["stimulus_id", "sentence"]].sample(
        min(5, (continuation_df["language"] == "en").sum()),
        random_state=0,
    )
)

display(
    continuation_df[continuation_df["language"] == "zh"][["stimulus_id", "sentence"]].sample(
        min(5, (continuation_df["language"] == "zh").sum()),
        random_state=0,
    )
)

Total stimuli: 128
language
en    64
zh    64
Name: count, dtype: int64


,stimulus_id,sentence
45,en-en_0-000045,A chef helped every tourist.
29,en-en_0-000029,A robot helped every tourist.
43,en-en_0-000043,A chef chased every doctor.
61,en-en_0-000061,A dog helped every tourist.
34,en-en_0-000034,A chef pushed every pirate.


,stimulus_id,sentence
109,zh-zh_0-000109,有一个厨师帮助了每个游客。
93,zh-zh_0-000093,有一个机器人帮助了每个游客。
107,zh-zh_0-000107,有一个厨师追了每个医生。
125,zh-zh_0-000125,有一只狗帮助了每个游客。
98,zh-zh_0-000098,有一个厨师推了每个海盗。


In [ ]:
# Serialize
continuation_df.to_csv(os.path.join(STIMULI_DIR,"stimuli.csv"), index=False)

with open(os.path.join(STIMULI_DIR, "stimuli.jsonl"), "w", encoding="utf-8") as f:
    for row in continuation_df.to_dict(orient="records"):
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print("Wrote stimuli.csv and stimuli.jsonl")

Wrote stimuli.csv and stimuli.jsonl


### Add Natural Language Continuations

In [139]:
EN_CONTINUATIONS = {
    "surface": " There was only one {subj}.",
    "inverse": " There were many {subj}.",
}

# Mandarin: keep equally short.
# Note: plural is usually implicit; "很多" is a decent lexical cue.
ZH_CONTINUATIONS = {
    "surface": " 只有一{cl}{subj}。",
    "inverse": " 有很多{cl}{subj}。",
}

In [140]:
def add_continuations(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for r in df.itertuples(index=False):
        base = r._asdict()

        if base["language"] == "en":
            # naive pluralization: add "s"
            # If you care about irregular plurals later, add a map.
            subj_plural = base["subj"] + "s"
            cont_map = {
                "surface": EN_CONTINUATIONS["surface"].format(subj=base["subj"]),
                "inverse": EN_CONTINUATIONS["inverse"].format(subj=subj_plural),
            }

        elif base["language"] == "zh":
            cl = ZH_CLASSIFIER.get(base["subj"], "个")
            cont_map = {
                "surface": ZH_CONTINUATIONS["surface"].format(cl=cl, subj=base["subj"]),
                "inverse": ZH_CONTINUATIONS["inverse"].format(cl=cl, subj=base["subj"]),
            }
        else:
            raise ValueError(f"Unknown language: {base['language']}")

        for cont_type, cont_text in cont_map.items():
            ex = dict(base)
            ex["continuation_type"] = cont_type            # "surface" or "inverse"
            ex["continuation_text"] = cont_text            # the thing you'll score
            ex["full_text"] = base["sentence"] + cont_text # convenient for debugging
            rows.append(ex)

    return pd.DataFrame(rows)



In [ ]:
df_cont = add_continuations(continuation_df)

if "concept_id" not in df_cont.columns:
    concept_series = df_cont["subj"] + "|" + df_cont["obj"] + "|" + df_cont["verb"]
    df_cont.insert(1, "concept_id", concept_series)

In [142]:
df_cont.to_csv(os.path.join(STIMULI_DIR, "stimuli_with_continuations.csv"), index=False)

with open(os.path.join(STIMULI_DIR, "stimuli_with_continuations.jsonl"), "w", encoding="utf-8") as f:
    for row in df_cont.to_dict(orient="records"):
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print("Wrote stimuli_with_continuations.csv and stimuli_with_continuations.jsonl")

Wrote stimuli_with_continuations.csv and stimuli_with_continuations.jsonl


## Step 2: Evaluate Log Probs of Continuations

In [2]:
import numpy as np
import json
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

/Users/kyleng/py_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load in Continuations

In [3]:
JSONL_PATH = "stimuli/stimuli_with_continuations.jsonl"

def read_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                yield json.loads(line)

In [4]:
rows = list(read_jsonl(JSONL_PATH))
continuation_df = pd.DataFrame(rows)

### Load Model and Tokenizer

In [5]:
MODEL_NAME = "gpt2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if (DEVICE == "cuda") else torch.float32

In [6]:
BATCH_SIZE = 16

In [7]:
OUT_CSV = "logprob_surface_vs_inverse.csv"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=DTYPE
).to(DEVICE)

model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Calculate Continuation Log Prob
1) Tokenize Prompt and Continuation

2) Calculate Log Probs for prompt + continuation

3) Calculate continuation log probs: [a] get log prob of each next token from [0,T-1]; [b] sum log probs
- Given a prompt of length T, log_prob[:, :T-1, :] gives the probability of the first [0,T-1] tokens

```

a shark ate every pirate
                    ^ stop here; we already know the probability of this word
```

```python
for b in range(B):
    for t in range(T):
        target_log_probs[b,t] = shifted_log_probs[b, t, target_tokens[b,t]]
```


In [10]:
def get_log_probs(prompts: list[str], continuations: list[str]) -> list[dict]:
    # 1) Tokenize Prompt and Continuation
    enc_base_prompts = tokenizer(
        prompts,
        return_tensors="pt",

        # Handle different length prompts
        padding=True,
        truncation=True,

        # Avoid [EOS]/[BOS] from being inserted
        add_special_tokens=False
    )

    full_prompts = [p + c for p, c in zip(prompts, continuations)]
    enc_full_prompts = tokenizer(
        full_prompts,
        return_tensors="pt",

        # Handle different length prompts
        padding=True,
        truncation=True,

        # Avoid [EOS]/[BOS] from being inserted
        add_special_tokens=False
    )

    input_ids = enc_full_prompts["input_ids"].to(DEVICE)
    attention_mask = enc_full_prompts["attention_mask"].to(DEVICE)

    # 2) Calculate logProbs for prompt + continuation
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = out.logits  # [B, T, V]
    log_probs = torch.log_softmax(logits, dim=-1)  # [B, T, V]

    # 3) Calculate continuation log probs: [a] get log prob of each next token from [0,T-1]; [b] sum log probs
    target_tokens = input_ids[:, 1:]  # [B, T-1]
    shifted_log_probs = log_probs[:, :-1, :]  # [B, T-1, V]

    # Select the logProb for the selected token
    target_log_probs = torch.gather(
        input=shifted_log_probs,
        dim=-1,  # Select the log_prob for the selected prompt token in vocab
        index=target_tokens.unsqueeze(-1)  # [B, T-1, 1]
    ).squeeze(-1)  # [B, T-1]

    base_prompt_lens = enc_base_prompts["attention_mask"].sum(dim=1).tolist()
    full_prompt_lens = enc_full_prompts["attention_mask"].sum(dim=1).tolist()

    # The logProbs for the continuation live at (inclusive)
    # logits[m-1:n-2] -> logits[m-1:L-2] -> target_log_probs[m-1:L-1] since we already removed one token in the shift
    B, _ = target_log_probs.shape

    cont_log_probs_list = []

    for b in range(B):
        base_prompt_length = base_prompt_lens[b]
        full_prompt_length = full_prompt_lens[b]

        cont_log_probs = target_log_probs[b,
                                          base_prompt_length-1:full_prompt_length-1]
        cont_log_probs_sum = cont_log_probs.sum().item()
        cont_log_probs_mean = cont_log_probs.mean().item()
        n_cont_tokens = full_prompt_length - base_prompt_length  # Sanity Check

        cont_log_probs_list.append(
            {"cont_log_probs_sum": cont_log_probs_sum,
                "cont_log_probs_mean": cont_log_probs_mean,
                "n_cont_tokens": n_cont_tokens
             }
        )
        # debugging
        # print(base_prompt_length, full_prompt_length)
        # print(attention_mask[b].shape, attention_mask[b])
        # print(target_log_probs[b].shape, target_log_probs[b])
        # print(cont_log_probs.shape,cont_log_probs)
        # print("="*10)
    return cont_log_probs_list

In [11]:
def process_log_probs(df: pd.DataFrame, b_sz: int = BATCH_SIZE):
    df = df.reset_index(drop=True).copy() # Pandas safety reasons
    all_stats = []

    for start in range(0, len(df), b_sz):
        end = min(start + b_sz, len(df))
        batch = continuation_df.iloc[start:end]

        stats = get_log_probs(
            prompts=batch["sentence"].tolist(),
            continuations=batch["continuation_text"].tolist()
        )

        all_stats += stats

    stats_df = pd.DataFrame(all_stats)    

    return pd.concat([df, stats_df], axis=1)

In [28]:
# AI Generated pivot table magic to get the metrics
def collapse_surface_inverse(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df["continuation_type"].isin(["surface", "inverse"])].copy()

    num_cols  = ["cont_log_probs_sum", "cont_log_probs_mean", "n_cont_tokens"]
    text_cols = ["continuation_text", "full_text"]

    # metadata that is shared across types
    meta_cols = [
        c for c in df.columns
        if c not in (["continuation_type"] + num_cols + text_cols)
    ]
    meta = df.groupby("stimulus_id", as_index=False)[meta_cols].first()

    # --- 1) pivot numeric columns (keeps float/int dtypes) ---
    wide_num = (
        df[["stimulus_id", "continuation_type"] + num_cols]
        .pivot(index="stimulus_id", columns="continuation_type", values=num_cols)
    )
    wide_num.columns = [f"{col}_{ctype}" for col, ctype in wide_num.columns]
    wide_num = wide_num.reset_index()

    # --- 2) pivot text columns (object dtype is fine here) ---
    wide_text = (
        df[["stimulus_id", "continuation_type"] + text_cols]
        .pivot(index="stimulus_id", columns="continuation_type", values=text_cols)
    )
    wide_text.columns = [f"{col}_{ctype}" for col, ctype in wide_text.columns]
    wide_text = wide_text.reset_index()

    # merge everything
    out = meta.merge(wide_num, on="stimulus_id", how="left") \
              .merge(wide_text, on="stimulus_id", how="left")

    # deltas + ratios (now safe: numeric dtypes)
    out["delta_sum"]  = out["cont_log_probs_sum_inverse"]  - out["cont_log_probs_sum_surface"]
    out["delta_mean"] = out["cont_log_probs_mean_inverse"] - out["cont_log_probs_mean_surface"]
    out["ratio_sum"]  = np.exp(out["delta_sum"])
    out["ratio_mean"] = np.exp(out["delta_mean"])

    out["preference_mean"] = np.select(
        [out["delta_mean"] > 0, out["delta_mean"] < 0],
        ["inverse", "surface"],
        default="tie"
    )
    out["preference_sum"] = np.select(
        [out["delta_sum"] > 0, out["delta_sum"] < 0],
        ["inverse", "surface"],
        default="tie"
    )

    return out

### Save Outputs
- Add model name as a column

In [46]:
from pathlib import Path
RESULTS_DIR = Path("./results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

In [47]:
import re
safe_model = re.sub(r"[^\w\-\.]", "_", MODEL_NAME)

In [48]:
scored_long = process_log_probs(continuation_df, b_sz=BATCH_SIZE)
scored_long.insert(0,"model", safe_model)

scored_wide = collapse_surface_inverse(scored_long)

In [49]:
scored_wide

,model,stimulus_id,concept_id,language,template_id,subj,obj,verb,sentence,cont_log_probs_sum_inverse,...,continuation_text_inverse,continuation_text_surface,full_text_inverse,full_text_surface,delta_sum,delta_mean,ratio_sum,ratio_mean,preference_mean,preference_sum
0,gpt2,en-en_0-000000,shark|pirate|ate,en,en_0,shark,pirate,ate,A shark ate every pirate.,-16.671429,...,There were many sharks.,There was only one shark.,A shark ate every pirate. There were many sharks.,A shark ate every pirate. There was only one s...,-1.176224,-0.751752,0.308441,0.471540,surface,surface
1,gpt2,en-en_0-000001,shark|pirate|helped,en,en_0,shark,pirate,helped,A shark helped every pirate.,-16.237549,...,There were many sharks.,There was only one shark.,A shark helped every pirate. There were many s...,A shark helped every pirate. There was only on...,-0.826022,-0.678922,0.437787,0.507163,surface,surface
2,gpt2,en-en_0-000002,shark|pirate|pushed,en,en_0,shark,pirate,pushed,A shark pushed every pirate.,-16.001699,...,There were many sharks.,There was only one shark.,A shark pushed every pirate. There were many s...,A shark pushed every pirate. There was only on...,-0.633678,-0.639003,0.530636,0.527818,surface,surface
3,gpt2,en-en_0-000003,shark|pirate|chased,en,en_0,shark,pirate,chased,A shark chased every pirate.,-15.284822,...,There were many sharks.,There was only one shark.,A shark chased every pirate. There were many s...,A shark chased every pirate. There was only on...,-0.457541,-0.585751,0.632838,0.556688,surface,surface
4,gpt2,en-en_0-000004,shark|student|ate,en,en_0,shark,student,ate,A shark ate every student.,-16.757080,...,There were many sharks.,There was only one shark.,A shark ate every student. There were many sha...,A shark ate every student. There was only one ...,-1.976831,-0.888041,0.138507,0.411461,surface,surface
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,gpt2,zh-zh_0-000123,狗|医生|追,zh,zh_0,狗,医生,追,有一只狗追了每个医生。,-32.516186,...,有很多只狗。,只有一只狗。,有一只狗追了每个医生。 有很多只狗。,有一只狗追了每个医生。 只有一只狗。,-12.770380,-1.160944,0.000003,0.313191,surface,surface
124,gpt2,zh-zh_0-000124,狗|游客|吃,zh,zh_0,狗,游客,吃,有一只狗吃了每个游客。,-32.146820,...,有很多只狗。,只有一只狗。,有一只狗吃了每个游客。 有很多只狗。,有一只狗吃了每个游客。 只有一只狗。,-13.137228,-1.194293,0.000002,0.302918,surface,surface
125,gpt2,zh-zh_0-000125,狗|游客|帮助,zh,zh_0,狗,游客,帮助,有一只狗帮助了每个游客。,-33.794079,...,有很多只狗。,只有一只狗。,有一只狗帮助了每个游客。 有很多只狗。,有一只狗帮助了每个游客。 只有一只狗。,-11.615507,-1.055955,0.000009,0.347860,surface,surface
126,gpt2,zh-zh_0-000126,狗|游客|推,zh,zh_0,狗,游客,推,有一只狗推了每个游客。,-31.862911,...,有很多只狗。,只有一只狗。,有一只狗推了每个游客。 有很多只狗。,有一只狗推了每个游客。 只有一只狗。,-12.681486,-1.152862,0.000003,0.315732,surface,surface


In [ ]:
scored_long.to_json(
    RESULTS_DIR / f"scored_long_{safe_model}.jsonl",
    orient="records",
    lines=True
)

scored_wide.to_json(
    RESULTS_DIR / f"scored_wide_{safe_model}.jsonl",
    orient="records",
    lines=True
)